<a href="https://colab.research.google.com/github/Carusof24/FFNSampling/blob/main/Model_and_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feedfoward

Viene definita una rete neurale feedforward con tre layer completamente connessi, usando funzioni di attivazione ReLU nei layer nascosti e Sigmoid nell’output.


In [ ]:
import torch
import torch.nn as nn

# Default precision
torch.set_default_dtype(torch.float64)

# -------------------- #
# Feedforward NN Model #
# -------------------- #
class FeedforwardNet(nn.Module):
    def __init__(self, input_dim=100, hidden_dim=100, output_dim=10):
        super(FeedforwardNet, self).__init__()
        # First hidden layer
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        # Secondo hidden layer
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        # Layer di output
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        #these are fully connected layers
        # Activaction function
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)   # Attivazione ReLU sul primo hidden layer
        x = self.fc2(x)
        x = self.relu(x)   # Attivazione ReLU
        x = self.fc3(x)
        x = self.sigmoid(x)  # Sigmoid per l'output
        return x

#wrapper is a class that encapsulates another class
# NNModel, this class provides a wrapper around the FFN class to manage its weights
class NNModel():
    def __init__(self, NN, device='cpu', f=None):
        self.NN = NN
        self.device = device if ('cuda' in device) and torch.cuda.is_available() else 'cpu'
        if f:
            self.load(f)
        else:
            self._to_device()
            self._init_weights()

    def _init_weights(self):
        self.weights = {name: param for name, param in self.NN.named_parameters() if param.requires_grad}

    def copy(self, grad=False):
        if not grad:
            wcopy = {name: self.weights[name].detach().clone() for name in self.weights}
        else:
            wcopy = {name: self.weights[name].grad.detach().clone() for name in self.weights}
        return wcopy

    def set_weights(self, wnew):
        assert all(name in self.weights for name in wnew), f"NNModel.set_weights(): invalid layer found in wnew. Allowed values: {list(self.weights.keys())}"
        for name, new_param in wnew.items():
            for pname, param in self.NN.named_parameters():
                if pname == name:
                    param.data = new_param.detach().clone()
        self._init_weights()

    def load(self, f):
        with open(f, 'rb') as ptf:
            self.NN.load_state_dict(torch.load(ptf, map_location=torch.device(self.device)))
        self._to_device()
        self._init_weights()

    def save(self, f):
        with open(f, 'wb') as ptf:
            torch.save(self.NN.state_dict(), ptf)

    def _to_device(self):
        if 'cuda' in self.device:
            self.NN.to(self.device)

In [ ]:
##Creiamo un dizionario dei pesi della rete, e specificamente i pesi del primo layer (fc1.weight) vengono sostituiti con valori casuali generati da una distribuzione normale.
#I pesi modificati vengono quindi reinseriti nel modello tramite il metodo set_weights.
#Il codice assicura che i pesi aggiornati nel modello corrispondano a quelli salvati nel wrapper NNModel.
#si verifica che i pesi attuali della rete coincidano con quelli memorizzati nel dizionario dei pesi.
#La verifica è effettuata confrontando i pesi presenti nel modello e quelli nel dizionario dopo il reset.
#il codice dimostra come gestire la modifica, il salvataggio e il ripristino dei pesi in una rete neurale implementata
# # Feedfoward

# Default precision
torch.set_default_dtype(torch.float64)

# parameters:
input_dim = 100
hidden_dim = 100
output_dim = 10

# Initialize the neural network
net = FeedforwardNet(input_dim, hidden_dim, output_dim)
model = NNModel(net)


# Get the initial weights
initial_weights = model.copy()


# Modify the weights
modified_weights = model.copy()

# change the weight of the first layer
for name, param in modified_weights.items():
    if name == 'fc1.weight':
        param.data = torch.randn_like(param.data)
        break


# Set the modified weights back into the model
model.set_weights(modified_weights)


# Verify that the weights in the NN and the weights stored in model.weights are the same
for name, param in model.NN.named_parameters():
    if name in modified_weights:
      print(f"Layer: {name}")
      print("Difference between model.weights and NN parameters:", torch.equal(model.weights[name], param.data))

# Reset weights to initial values (example)
model.set_weights(initial_weights)

# Verify that the weights have been reset correctly
for name, param in model.NN.named_parameters():
    if name in initial_weights:
      print(f"Layer: {name}")
      print("Difference between model.weights and NN parameters after reset:", torch.equal(model.weights[name], param.data))


Layer: fc1.weight
Difference between model.weights and NN parameters: True
Layer: fc1.bias
Difference between model.weights and NN parameters: True
Layer: fc2.weight
Difference between model.weights and NN parameters: True
Layer: fc2.bias
Difference between model.weights and NN parameters: True
Layer: fc3.weight
Difference between model.weights and NN parameters: True
Layer: fc3.bias
Difference between model.weights and NN parameters: True
Layer: fc1.weight
Difference between model.weights and NN parameters after reset: True
Layer: fc1.bias
Difference between model.weights and NN parameters after reset: True
Layer: fc2.weight
Difference between model.weights and NN parameters after reset: True
Layer: fc2.bias
Difference between model.weights and NN parameters after reset: True
Layer: fc3.weight
Difference between model.weights and NN parameters after reset: True
Layer: fc3.bias
Difference between model.weights and NN parameters after reset: True


In [ ]:
#Manage e manipulate the weights of neural network using NNModel.
#Make changes to the weights (simulating a "move" in a search or optimization process).
#Revert changes to the weights if needed (like rejecting a move).
# Inizializza la rete e il wrapper
net = FeedforwardNet()
nn_model = NNModel(net)

# Salvataggio dei pesi iniziali
w_initial = nn_model.copy()

# Stampa dei pesi iniziali del layer fc1
print("Pesi iniziali di fc1.weight:")
print(nn_model.weights['fc1.weight'])

# Simulazione di una mossa, modifica del peso fc1.weight
wnew = nn_model.copy()
wnew['fc1.weight'] = wnew['fc1.weight'] + 1.0

# Applica la mossa proposta
nn_model.set_weights(wnew)
print("\nPesi dopo la mossa proposta di fc1.weight:")
print(nn_model.weights['fc1.weight'])

# Supponiamo di voler rifiutare la mossa: ripristino della configurazione precedente
nn_model.set_weights(w_initial)
print("\nPesi dopo il ripristino (rifiuto della mossa) di fc1.weight:")
print(nn_model.weights['fc1.weight'])

Pesi iniziali di fc1.weight:
Parameter containing:
tensor([[ 0.0961, -0.0802,  0.0636,  ...,  0.0916, -0.0843, -0.0768],
        [ 0.0774,  0.0070,  0.0699,  ...,  0.0918, -0.0963, -0.0100],
        [-0.0607,  0.0815,  0.0096,  ...,  0.0590,  0.0252,  0.0582],
        ...,
        [-0.0222,  0.0812, -0.0688,  ...,  0.0828,  0.0650, -0.0279],
        [-0.0128,  0.0341,  0.0301,  ..., -0.0096, -0.0427, -0.0035],
        [-0.0952,  0.0218,  0.0412,  ..., -0.0221,  0.1000,  0.0964]],
       requires_grad=True)

Pesi dopo la mossa proposta di fc1.weight:
Parameter containing:
tensor([[1.0961, 0.9198, 1.0636,  ..., 1.0916, 0.9157, 0.9232],
        [1.0774, 1.0070, 1.0699,  ..., 1.0918, 0.9037, 0.9900],
        [0.9393, 1.0815, 1.0096,  ..., 1.0590, 1.0252, 1.0582],
        ...,
        [0.9778, 1.0812, 0.9312,  ..., 1.0828, 1.0650, 0.9721],
        [0.9872, 1.0341, 1.0301,  ..., 0.9904, 0.9573, 0.9965],
        [0.9048, 1.0218, 1.0412,  ..., 0.9779, 1.1000, 1.0964]],
       requires_grad=Tru

##Dataset
Il dataset generato è costituito da vettori di spin ±1 di dimensione 100, con 10 diverse configurazioni iniziali.
Ogni vettore subisce una perturbazione applicando una probabilità di flip del 10%, invertendo casualmente alcuni spin per introdurre rumore.
L’output associato è una label one-hot, ovvero un vettore di 10 elementi con un unico valore pari a 1, che identifica quale vettore originale è stato usato come base per il campione modificato.
Questa rappresentazione suggerisce un problema di classificazione in cui la rete neurale deve riconoscere la configurazione iniziale da cui proviene ogni input perturbato.

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset

def flip_spin(vector, p=0.1):
    """
    Data una sequenza di spin (±1), inverte ciascun elemento con probabilità p.
    """
    flip_mask = np.random.rand(vector.shape[0]) < p  # Quali spin invertire
    vector_flipped = vector.copy()
    vector_flipped[flip_mask] *= -1  # Inversione degli spin selezionati
    return vector_flipped

def generate_dataset(n_vectors=10, vector_dim=100, p_flip=0.1):
    """
    Genera un dataset di n_vectors vettori, ognuno di 10dimensionale
    Ogni vettore contiene spin ±1, e viene flippato con probabilità p_flip.
    """
    dataset = []
    labels = []

    for k in range(n_vectors):
        s_k = np.random.choice([-1, 1], size=vector_dim)  # Vettore iniziale
        s_i = flip_spin(s_k, p_flip)  # Flippo
        dataset.append(s_i)

        # Creazione della label one-hot (10 classi)
        label = np.zeros(n_vectors)
        label[k] = 1
        labels.append(label)

    return np.array(dataset), np.array(labels)

class SpinDataset(Dataset):
    """
    Dataset compatibile con PyTorch, restituisce coppie (input, target).
    """
    def __init__(self, n_vectors=10, vector_dim=100, p_flip=0.1):
        X, y = generate_dataset(n_vectors, vector_dim, p_flip)
        self.X = torch.tensor(X, dtype=torch.float64)  # Input
        self.y = torch.tensor(y, dtype=torch.float64)  # One-hot label

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Test dataset
if __name__ == "__main__":
    dataset = SpinDataset(n_vectors=10, vector_dim=100, p_flip=0.1)
    print("Esempio di input:", dataset[0][0])  # Un vettore di spin
    print("Esempio di label:", dataset[3][1])  # La label one-hot

Esempio di input: tensor([ 1.,  1., -1., -1.,  1., -1., -1., -1.,  1., -1., -1., -1.,  1.,  1.,
        -1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,
         1.,  1.,  1.,  1., -1.,  1.,  1.,  1., -1.,  1.,  1., -1.,  1.,  1.,
         1., -1.,  1., -1., -1., -1., -1., -1.,  1.,  1., -1.,  1., -1.,  1.,
        -1.,  1., -1.,  1., -1.,  1., -1., -1.,  1.,  1.,  1.,  1., -1.,  1.,
         1.,  1.,  1., -1.,  1., -1., -1.,  1.,  1., -1.,  1., -1., -1., -1.,
         1.,  1., -1., -1., -1.,  1., -1.,  1., -1.,  1., -1., -1., -1., -1.,
        -1., -1.])
Esempio di label: tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])


# training with ADAM

# training with SGD


